In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [16]:
a = np.arange(27).reshape((3, 3,3))

c=a.reshape((-1,9)).astype(np.float32)
print("reshape 的-1是用来给函数降维的")
print(c)
print(c.shape)

reshape 的-1是用来给函数降维的
[[  0.   1.   2.   3.   4.   5.   6.   7.   8.]
 [  9.  10.  11.  12.  13.  14.  15.  16.  17.]
 [ 18.  19.  20.  21.  22.  23.  24.  25.  26.]]
(3, 9)


In [17]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
print("使三维矩阵变成二维的矩阵")

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
使三维矩阵变成二维的矩阵


In [19]:

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :]) #每次取10000个数据进行训练
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels])) #初始化矩阵w，大小为784*10
  biases = tf.Variable(tf.zeros([num_labels]))                  #初始化偏置变量
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases       #y=w*x+b
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) #logits代表通过矩阵运算出来的数据结果，labels代表实际的标签
                                                                                     #通过softmax计算的结果后通过减小交叉熵来进行运算。           
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)#通过gradient descent来优化交叉熵。0.5代表的意思不确定，有可能是优化的速率。下降的速率 
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits) #softmax函数算出来的结果应该是一个矩阵
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  print("this step is finished")

this step is finished


In [21]:
num_steps = 801

def accuracy(predictions, labels):  #计算训练后的arruracy，argmax得出矩阵中的最大值，或者其对应的序号。axis=0代表每一列，axis=1代表每一行。
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))  #对于train_label，shape是200000,10。 因此我们现在需要的是每一行的
          / predictions.shape[0])                                            #最大值。因此用 np.argmax(labels, 1))

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):   #step代表优化的次数。相当于每个step只优化一步。意思是每一步中，对这10000个subset算出loss，然后根据loss改变
    # Run the computations. We tell .run() that we want to run the optimizer,     一次矩阵w中的系数和偏置b的值。自己选择计算的次数多少次优化。
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))            #这里只是代表每次只取train_data的前10000个数据进行prediction。
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 13.428686
Training accuracy: 13.9%
Validation accuracy: 16.6%
Loss at step 100: 2.276607
Training accuracy: 71.7%
Validation accuracy: 70.8%
Loss at step 200: 1.810206
Training accuracy: 75.4%
Validation accuracy: 73.5%
Loss at step 300: 1.574323
Training accuracy: 77.0%
Validation accuracy: 74.2%
Loss at step 400: 1.417476
Training accuracy: 77.9%
Validation accuracy: 74.6%
Loss at step 500: 1.301884
Training accuracy: 78.4%
Validation accuracy: 75.0%
Loss at step 600: 1.211475
Training accuracy: 79.1%
Validation accuracy: 75.2%
Loss at step 700: 1.137780
Training accuracy: 79.3%
Validation accuracy: 75.3%
Loss at step 800: 1.076002
Training accuracy: 79.6%
Validation accuracy: 75.5%
Test accuracy: 82.8%


In [23]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  print("step fini")

step fini


In [24]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels} #tf_train_dataset 给placeholder里面赋值
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print(offset)
      print(train_labels.shape[0] - batch_size)
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print('')
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
0
199872
Minibatch loss at step 0: 23.313967
Minibatch accuracy: 4.7%
Validation accuracy: 12.2%

64000
199872
Minibatch loss at step 500: 1.726117
Minibatch accuracy: 76.6%
Validation accuracy: 76.4%

128000
199872
Minibatch loss at step 1000: 1.233582
Minibatch accuracy: 78.1%
Validation accuracy: 77.0%

192000
199872
Minibatch loss at step 1500: 0.907023
Minibatch accuracy: 80.5%
Validation accuracy: 78.3%

56128
199872
Minibatch loss at step 2000: 0.963275
Minibatch accuracy: 82.8%
Validation accuracy: 78.6%

120128
199872
Minibatch loss at step 2500: 0.820735
Minibatch accuracy: 79.7%
Validation accuracy: 78.7%

184128
199872
Minibatch loss at step 3000: 0.975867
Minibatch accuracy: 73.4%
Validation accuracy: 78.8%

Test accuracy: 85.6%


In [30]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  
  
  # Training computation.
  logits = tf.matmul(hidden1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)


# In[23]:

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print ("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print ("Minibatch loss at step", step, ":", l)
      print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print ("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0 : 371.335
Minibatch accuracy: 3.9%
Validation accuracy: 28.2%
Minibatch loss at step 500 : 13.543
Minibatch accuracy: 82.8%
Validation accuracy: 79.4%
Minibatch loss at step 1000 : 4.42868
Minibatch accuracy: 84.4%
Validation accuracy: 81.7%
Minibatch loss at step 1500 : 8.40253
Minibatch accuracy: 79.7%
Validation accuracy: 81.7%
Minibatch loss at step 2000 : 8.97501
Minibatch accuracy: 78.1%
Validation accuracy: 81.0%
Minibatch loss at step 2500 : 4.47768
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 3000 : 3.13789
Minibatch accuracy: 77.3%
Validation accuracy: 82.4%
Test accuracy: 88.6%
